In [1]:
import sys
import os

parent_path = os.path.abspath("..")  # Get absolute path of parent directory
if parent_path not in sys.path:  
    sys.path.append(parent_path)  # Add only if it's not already there
    
from dotenv import load_dotenv
load_dotenv("../.env")


import pandas as pd
import chromadb
import openai

from src.models.chromadb_functions import *  

### Read Excel file 

In [9]:
question_answer_df = pd.read_excel("../data/question_answer/100 câu hỏi.xlsx")

print(question_answer_df)

print("Excel Length: ", len(question_answer_df))

    Mã câu hỏi                                            Câu Hỏi  \
0            1  Hướng dẫn cho tôi cách sử dụng phân bón của Ph...   
1            2  Hướng dẫn cho tôi cách bón và liều lượng bón c...   
2            3  Hướng dẫn cho tôi cách bón và liều lượng bón c...   
3            4  Hướng dẫn cho tôi cách bón và liều lượng bón c...   
4            5  Hướng dẫn cho tôi cách bón và liều lượng bón c...   
..         ...                                                ...   
95          96  Bón phân cho cây lúa vào mùa mưa và mùa khô có...   
96          97  Khi nào là thời điểm bón phân tốt nhất cho cây...   
97          98         Có nên bón phân khi cây đang ra hoa không?   
98          99  Ngay sau khi thu hoạch cần phải bón phân liền ...   
99         100  Có thể bón phân trong giai đoạn cây bị sâu bện...   

                                          Câu trả lời  
0   Dạ. bà con vui lòng nói tên sản phẩm cụ thể hơ...  
1   Cách hướng dẫn sử dụng phân bón u rê hạt đục n...  
2  

### Get ChromaDB Collection

In [2]:
collection_path     = "../data/vector_database"
collection_name     = os.getenv("CHROMADB_COLLECTION_NAME")
embedding_func      = define_embedding_function( api_key = os.getenv("OPENAI_API_KEY"), model_name = os.getenv("EMBEDDING_MODEL"))
similarity_method   = os.getenv("SIMILARITY_METHOD")

In [3]:
chromadb_collection = get_chroma_collection(collection_path, collection_name, embedding_func, similarity_method)

### Load Questions into ChromaDB
- With index in metadata and index for tracking

In [10]:
list_base_question  = list(question_answer_df["Câu Hỏi"])
list_question_no    = list(question_answer_df["Mã câu hỏi"])

In [11]:
chromadb_collection.add(
    documents=list_base_question,
    metadatas = [{"video": f"{i}.mp4"} for i in list_question_no],
    ids = [str(i) for i in list_question_no]
)

### Update question in ChromaDB using index

In [ ]:
chromadb_collection.update(
    ids=[str(i) for i in list_question_no],
    metadatas=[{"video": f"{i}.mp4"} for i in list_question_no]
)

### Query Similar Question in ChromaDB

In [4]:
results = chromadb_collection.query(
    query_texts=["Làm sao để chăm sóc cây ăn trái?"], # Chroma will embed this for you
    n_results=1 # how many results to return
)

print(results)

{'ids': [['2']], 'embeddings': None, 'documents': [['Cây nhãn chăm sóc như thế nào ?']], 'uris': None, 'data': None, 'metadatas': [[{'video': '2.mp4'}]], 'distances': [[0.35213560517967557]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


### Get all Data from Collection

In [12]:
data = chromadb_collection.get()
data

{'ids': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '40',
  '41',
  '42',
  '43',
  '44',
  '45',
  '46',
  '47',
  '48',
  '49',
  '50',
  '51',
  '52',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '63',
  '64',
  '65',
  '66',
  '67',
  '68',
  '69',
  '70',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '83',
  '84',
  '85',
  '86',
  '87',
  '88',
  '89',
  '90',
  '91',
  '92',
  '93',
  '94',
  '95',
  '96',
  '97',
  '98',
  '99',
  '100'],
 'embeddings': None,
 'documents': ['Hướng dẫn cho tôi cách sử dụng phân bón của Phân Bón Cà Mau',
  'Hướng dẫn cho tôi cách bón và liều lượng bón của phân bón Đạm Cà Mau',
  'Hướng dẫn cho tôi cách b

### Delete Data in ChromaDB

In [7]:
chromadb_collection.delete(
    ids=["1", "2"]
)